# Fit teaching models to behavioral data
Natalia Vélez & Alicia Chen, November 2021

In [1]:
%matplotlib inline

import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval as eval_tuple

sys.path.append('..')
from utils import read_json, int_extract

sns.set_context('talk')
sns.set_style('white')

grid_size = 6 # exp parameter

Load teaching problems:

In [2]:
problems = read_json('inputs/problems.json')
problems[0]

{'A': [[0, 0, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0],
  [1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1],
  [0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 0, 0]],
 'B': [[1, 1, 1, 0, 0, 0],
  [1, 1, 1, 0, 0, 0],
  [1, 1, 1, 1, 0, 0],
  [0, 0, 1, 1, 0, 0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0]],
 'C': [[0, 0, 0, 1, 1, 1],
  [0, 0, 0, 1, 1, 1],
  [0, 0, 1, 1, 1, 1],
  [0, 0, 1, 1, 0, 0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0]],
 'D': [[0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 1, 1, 0, 0],
  [1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1]]}

Load exclusions:

In [3]:
excluded = np.loadtxt('../1_preprocessing/outputs/excluded_participants.txt', dtype=str)
excluded = [int_extract('[0-9]+', s) for s in excluded]
print(excluded)

[3, 17]


Load teaching data:

In [86]:
def read_examples(e):
    coords = eval_tuple(e)
    idx = np.ravel_multi_index(coords, (6,6))
    
    return idx

human_df = pd.read_csv('outputs/teaching_behavior.csv')
human_df = human_df.drop(columns=['onset', 'order', 'rating']) # drop columns that are irrelevant for model
human_df = human_df[~pd.isna(human_df.example)] # exclude trials where teachers failed to respond
human_df['example'] = human_df.example.apply(read_examples)

human_df.head()

,subject,run,block_idx,ex_idx,problem,example
0,1,1,0,0,22,8
1,1,1,0,1,22,27
3,1,1,1,0,18,3
4,1,1,1,1,18,32
5,1,1,1,2,18,7


## Sampling methods

Helper function: Convert problem into dataframe of coordinates x hypotheses

In [173]:
def problem_df(prob_idx):
    prob = problems[prob_idx]
    hypotheses = np.array(list(prob.values())) # read hypothesis space
    hypotheses_flat = np.reshape(hypotheses, (hypotheses.shape[0], hypotheses.shape[1]*hypotheses.shape[2])) #  flatten 3d => 2d array

    ### reshape into dataframe of coordinates x hypotheses
    df = pd.DataFrame(hypotheses_flat).stack().rename_axis(['hypothesis', 'idx']).reset_index(name='val')
    
    # name hypotheses
    df['hypothesis'] = pd.Categorical(df.hypothesis)
    df['hypothesis'] = df.hypothesis.cat.rename_categories(['A', 'B', 'C', 'D'])
    df['hypothesis'] = df['hypothesis'].astype('object')
    
    # spread each hypothesis into its own column
    df = df.pivot(index='idx', columns='hypothesis', values='val')
    df = df[df.sum(axis=1) > 0]
    
    return df

In [177]:
problem_df(20) # let's test it out!

hypothesis,A,B,C,D
idx,,,,
9,1,1,1,0
10,0,1,0,0
14,1,1,1,0
15,1,1,1,0
16,0,1,0,0
19,1,1,1,1
20,1,1,1,1
21,1,1,1,1
22,0,1,0,0


### a) Strong sampling

Define the sampling method:

In [178]:
def strong_sampling(prob_idx, past_examples=[]):
    '''
    Input: Index of problem (as it appears in "problems" list)
    Output: Dataframe of the probability of selecting the data (idx), given the hypothesis (A, B, C, D)
    '''
    # find available examples
    all_examples = problem_df(prob_idx)
    if len(past_examples):
        available_examples = all_examples.drop(past_examples)
    else:
        available_examples = all_examples.copy()

    # select uniformly among available examples
    pD = available_examples.div(available_examples.sum(axis=0), axis=1)
    
    return pD

Main method: Compute likelihood of data under strong sampling

In [180]:
def fit_strong_sampling(group):
    model_outputs = []
    examples = []
    
    for _, row in group.iterrows():
        ex = row.example
        # likelihood of observed data, assuming strong sampling
        strong_pD = strong_sampling(row.problem, past_examples=examples)
        out = row.copy()
        out['strong_lik'] = strong_pD['A'].loc[ex]
        
        # learner's posterior belief given the data
        strong_pH = strong_pD.div(strong_pD.sum(axis=1), axis=0)
        out['strong_pTrue'] = strong_pH['A'].loc[ex]
        
        examples.append(ex)
        model_outputs.append(out)
    
    model_outputs = pd.DataFrame(model_outputs)
    
    return model_outputs

Loop through behavioral data:

In [181]:
strong_predictions = []
for name, group in human_df.groupby(['subject', 'run', 'block_idx']):
    pred = fit_strong_sampling(group)
    strong_predictions.append(pred)
    
strong_predictions = pd.concat(strong_predictions)
strong_predictions.head()

,subject,run,block_idx,ex_idx,problem,example,strong_lik,strong_pTrue
0,1.0,1.0,0.0,0.0,22.0,8.0,0.062500,0.305389
1,1.0,1.0,0.0,1.0,22.0,27.0,0.066667,0.302926
3,1.0,1.0,1.0,0.0,18.0,3.0,0.083333,0.586207
4,1.0,1.0,1.0,1.0,18.0,32.0,0.090909,0.592593
5,1.0,1.0,1.0,2.0,18.0,7.0,0.100000,0.315789


### b) Pedagogical sampling

Define the sampling method:

In [207]:
def pedagogical_sampling(prob_idx, past_examples=[], pH=None, nIter=100):
    
    # if this is not the first trial: drop past examples
    if len(past_examples):
        most_recent = np.intersect1d(pH.index, past_examples)
        pH = pH.drop(most_recent)
    # else: start from a uniform prior
    else:
        hypothesis_space = problem_df(prob_idx)
        uniform_prior = hypothesis_space.div(hypothesis_space.sum(axis=1), axis=0)
        pH = uniform_prior
        
    # ~ recursive reasoning ~
    for _ in range(nIter):
        pD = pH.div(pH.sum(axis=0), axis=1)
        pH = pD.div(pD.sum(axis=1), axis=0)
        
    return pD, pH

Main method: Compute likelihood of data under pedagogical sampling

In [210]:
def fit_pedagogical_sampling(group, nIter=20):
    model_outputs = []
    examples = []
    pH = None
    
    for _, row in group.iterrows():
        ex = row.example
        # likelihood of observed data, assuming strong sampling
        pedagogical_pD, pH = pedagogical_sampling(row.problem, past_examples=examples, pH=pH, nIter=nIter)
        out = row.copy()
        out['pedagogical_lik'] = pedagogical_pD['A'].loc[ex]
        
        # learner's posterior belief given the data
        out['pedagogical_pTrue'] = pH['A'].loc[ex]
        
        examples.append(ex)
        model_outputs.append(out)
    
    model_outputs = pd.DataFrame(model_outputs)
    
    return model_outputs

Loop through behavioral data:

In [219]:
pedagogical_predictions = []
for name, group in human_df.groupby(['subject', 'run', 'block_idx']):
    pred = fit_pedagogical_sampling(group, nIter=10)
    pedagogical_predictions.append(pred)
    
pedagogical_predictions = pd.concat(pedagogical_predictions)
pedagogical_predictions.head()

,subject,run,block_idx,ex_idx,problem,example,pedagogical_lik,pedagogical_pTrue
0,1.0,1.0,0.0,0.0,22.0,8.0,0.063933,0.319666
1,1.0,1.0,0.0,1.0,22.0,27.0,0.066489,0.315820
3,1.0,1.0,1.0,0.0,18.0,3.0,0.138764,0.589748
4,1.0,1.0,1.0,1.0,18.0,32.0,0.151758,0.607031
5,1.0,1.0,1.0,2.0,18.0,7.0,0.090584,0.339688


## Compare model predictions

Put both models together:

In [213]:
model_predictions = strong_predictions.merge(pedagogical_predictions)
model_predictions.head()

,subject,run,block_idx,ex_idx,problem,example,strong_lik,strong_pTrue,pedagogical_lik,pedagogical_pTrue
0,1.0,1.0,0.0,0.0,22.0,8.0,0.062500,0.305389,0.063933,0.319666
1,1.0,1.0,0.0,1.0,22.0,27.0,0.066667,0.302926,0.066488,0.315820
2,1.0,1.0,1.0,0.0,18.0,3.0,0.083333,0.586207,0.138764,0.589748
3,1.0,1.0,1.0,1.0,18.0,32.0,0.090909,0.592593,0.151758,0.607031
4,1.0,1.0,1.0,2.0,18.0,7.0,0.100000,0.315789,0.090584,0.339688


Likelihood ratios:

In [220]:
model_predictions['strong_loglik'] = np.log(model_predictions['strong_lik'])
model_predictions['pedagogical_loglik'] = np.log(model_predictions['pedagogical_lik'])

model_lik = model_predictions.groupby('subject')[['strong_loglik', 'pedagogical_loglik']].agg('sum').reset_index()
model_lik['best_model'] = np.where(model_lik['pedagogical_loglik'] > model_lik['strong_loglik'], 'pedagogical', 'strong')

In [222]:
model_lik

,subject,strong_loglik,pedagogical_loglik,best_model
0,1.0,-257.201687,-250.074259,pedagogical
1,2.0,-271.035399,-269.637825,pedagogical
2,3.0,-223.189324,-221.525373,pedagogical
3,4.0,-268.966008,-262.786910,pedagogical
4,5.0,-273.210967,-270.310751,pedagogical
5,6.0,-275.484565,-272.630569,pedagogical
6,7.0,-281.667921,-270.121261,pedagogical
7,8.0,-282.785951,-270.366681,pedagogical
8,9.0,-279.445378,-289.430955,strong
9,10.0,-265.667129,-262.156667,pedagogical
